In [ ]:
# Load the Pandas libraries with alias 'pd' 
import numpy as np
import pandas as pd 
import matplotlib.colors as Colors
import matplotlib.pyplot as plt
import os
import cv2
import sys
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
from scipy.stats import entropy
from os import listdir
from os.path import isfile, join
import math
from scipy.stats import chisquare
from scipy import stats
from itertools import product
import pickle
import random


np.set_printoptions(threshold=sys.maxsize)

def plotAllModelsObservables(models, coeffs):
    
    
    observables = loadObservables()
    
    numObservables = len(observables)
    nrows = math.ceil(numObservables/2)
    ncols = 2
    nrows =3
    ncols =3
    imageWidth = 6
    imageHeight = 5
    hspace = 0.5
    wspace = 0.5
    figureHeight = nrows * imageHeight + (nrows - 1) * hspace
    figureWidth = ncols * imageWidth + (ncols - 1) * wspace
    
    plt.rc('text', usetex=True)
    plt.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]    
    
    
    for C9, C10 in models:    
        index = 1
        fig, axs = plt.subplots(nrows,ncols)
        fig.subplots_adjust(left=None, right=None, bottom=None, top=0.90, hspace=hspace, wspace=wspace)  
        fig.set_figheight(figureHeight)
        fig.set_figwidth(figureWidth)
        fig.delaxes(axs[2,2])
        axs = np.reshape(axs,(-1))
        axis_index = 0
        allFits=[]
        for mo, eo in observables:
            
            #fig.suptitle(title, fontsize=18) 
            df = mo['data']
            de = eo['data']
            
            ax = axs[axis_index]            
            
            observableName = mo['name']
            
            ax.set_xlabel(r'\(\boldsymbol{q^2(GeV^2)}\)', fontsize=16)
            print(observableName)
            if(observableName == 'RKStar'):
                ytitle = 'R'
                ax.set_ylabel(r'\(\langle \boldsymbol{{{}_K^*}} \rangle\)'.format(ytitle), fontsize=16)
            elif(observableName=='BRK0mumu'):
                ytitle = 'B'
                ax.set_ylabel(r'\(\langle \boldsymbol{{\mathcal{B}(B^0\rightarrow K\mu\mu)}} \rangle\)', fontsize=16)
            elif(observableName=='BRK0Starmumu'):
                ytitle = 'B'
                ax.set_ylabel(r'\(\langle \boldsymbol{{\mathcal{B}(B^0\rightarrow K^{*0}\mu\mu)}} \rangle\)', fontsize=16)
            elif(observableName == 'P5'):
                ytitle = 'P'
                ax.set_ylabel(r'\(\langle \boldsymbol{{{}_5}} \rangle\)'.format(ytitle), fontsize=16)
            else:
                ax.set_ylabel(r'\(\langle \boldsymbol{{{}}} \rangle\)'.format(observableName), fontsize=16)
                
            computeModelObservable(df, C9, C10)
            if(len(coeffs) > 0):
                addQPoly(df,coeffs[axis_index])
                
            fit = computeModelFit(df, de)
            print(fit)
            print(coeffs[axis_index])
            allFits.append(fit)
            ax.set_title(r'\(\chi^2:{} \)'.format(round(fit,2)), fontsize=20)
            ax.set_xlim(0, max(df['BinHigh'].max(), de['BinHigh'].max()))
            ax.set_ylim([min(df['MinValue'].min(),de['MinValue'].min()), max(df['MaxValue'].max(),de['MaxValue'].max())])
            
            plotObservable(ax, df, 'orange', 'black', 1)
            plotObservable(ax, de, 'black', 'green', 0, True, True)
            index = index + 1 
            
            axis_index = axis_index + 1
        title = r'Model: C9: {}, C10: {}, \(\chi^2: {} \)'.format(round(C9,2), round(C10,2),round(np.array(allFits).sum(),2))         
        fig.suptitle(title, fontsize=24) 
        fig.savefig('observableImages/{}.svg'.format('AllObservables'))
    plt.show()
   
    
    
def expandObservableData(df):
    df['Widths'] = df['BinHigh'] - df['BinLow']
    df['Heights'] = df['ErrorPlus'] + df['ErrorMinus']  
    
       

def plotObservable(ax, df, color,kcolor, opacity, yerrorbar = False, onlyErrorBars = False):   
    patches = [Rectangle((x,y), w, h) for x,y,w,h in zip(df['BinLow'], df['MinValue'], df['Widths'], df['Heights'])]
    xerror = [df['Widths'] / 2, df['Widths'] / 2]    
    
    if(not onlyErrorBars):
        p = PatchCollection(patches, facecolor=Colors.to_rgba(color, opacity))
        ax.add_collection(p)       
    ax.errorbar(df['BinLow'] + df['Widths']/2, df['BinValue'], xerr=xerror, fmt='None', ecolor=Colors.to_rgba(kcolor, 1))
    if(yerrorbar):
        yerror = [df['Heights'] / 2, df['Heights'] / 2] 
        ax.errorbar(df['BinLow'] + df['Widths']/2, df['BinValue'], yerr=yerror, fmt='None', ecolor=Colors.to_rgba(kcolor, 1))


def computePoly(binCoeffs,c9,c10):
    a = binCoeffs[0]
    b = binCoeffs[1]*c10
    c = binCoeffs[2]*c10**2
    d = binCoeffs[3]*c9
    e = binCoeffs[4]*c10*c9
    f = binCoeffs[5]*c9**2
    return a + b + c  + d + e + f

def getBinValuesFreeCoeffs(df, c9, c10, observableCoeffs):     
    df['BinValue'] = [computePoly(binCoeffs,c9,c10) for binCoeffs in observableCoeffs]
    
        
def getBinValues(df, C9, C10):     
    df2 = df.copy();       
    df2['C10'] = df2['C10']*C10
    df2['C10^2'] = df2['C10^2']*C10**2
    df2['C9'] = df2['C9']*C9
    df2['C10*C9']=df2['C10*C9']*C10*C9
    df2['C9^2'] = df2['C9^2']*C9**2    
    df['BinValue'] = df['C0'] + df2['C10'] + df2['C10^2'] + df2['C9'] + df2['C10*C9'] + df2['C9^2']
    df['MinValue'] = df['BinValue'] - df['ErrorMinus']
    df['MaxValue'] = df['BinValue'] + df['ErrorPlus']

def getModelObservable(DATADIR, filename) :
    return getObservable(DATADIR, filename)

def getExpObservable(DATADIR, filename) :
    eo = getObservable(DATADIR, filename)
    df = eo['data']
    df['MinValue'] = df['BinValue'] - df['ErrorMinus']
    df['MaxValue'] = df['BinValue'] + df['ErrorPlus']
    return eo

def getObservable(DATADIR, filename):
    try:  
        df = pd.read_csv(os.path.join(DATADIR, filename))
        expandObservableData(df)
        fname = filename.split('.')[0]              
        return {'name': fname, 'data': df}
    except Exception as e:
        print('Exception',e)
        pass
    
def computeModelFit2(dfm, dfe):
    data = zip(dfm['BinValue'], dfe['BinValue'], (dfm['ErrorPlus']+dfm['ErrorMinus'])/2, (dfe['ErrorPlus']+dfe['ErrorMinus'])/2)
    binFits = [computeChiSquare(muModel, muExp, sigmaModel, sigmaExp) for muModel, muExp, sigmaModel, sigmaExp in data]      
    s = sum(binFits)
    return s
        
def computeChiSquare(muModel, muExp, sigmaModel, sigmaExp):   
    return (muExp - muModel)**2/(sigmaModel**2 + sigmaExp**2)
    

def getObservables(DATADIR, EXPDIR, names):
    observables = []
    for filename in names:
        mo = getModelObservable(DATADIR, filename)        
        eo = getExpObservable(EXPDIR, filename)
        observables.append([mo, eo])
    return observables
    
def getAllObservables():
    DATADIR = 'data/models/observables' 
    EXPDIR = 'data/experimental'     
    onlyfiles = [f for f in listdir(DATADIR) if isfile(join(DATADIR, f))]    
    return getObservables(DATADIR, EXPDIR, onlyfiles)   

def computeRandomModelsFit(observables):    
    models = getRandomModels()
    return computeModelsFit(models, observables)    

def getCoeffsFromModels(models):
    m = [[c9, c10] for c9, c10 in zip(models['C9'], models['C10'])]
    return m
    
def getModels():
    MODELSDIR = 'data/models'  
    mfilename = 'models1.txt'    
    models = pd.read_csv(os.path.join(MODELSDIR, mfilename))
    return models

def addQPoly(dfe, coeffs):
    dfe['q^2'] = (dfe['BinHigh'] + dfe['BinLow']) / 2
    
    dfe['Fq^2Net'] = coeffs[0]  + coeffs[1] * dfe['q^2'] + coeffs[2] * dfe['q^2']**2
    
    dfe['BinValueOld'] = dfe['BinValue']
    dfe['BinValue'] = dfe['BinValue'] + dfe['Fq^2Net']  
    dfe['BinValueDiff'] = dfe['BinValue'] - dfe['BinValueOld']
    dfe['MinValue'] = dfe['BinValue'] - dfe['ErrorMinus']
    dfe['MaxValue'] = dfe['BinValue'] + dfe['ErrorPlus']
    
        
def computeModelObservable(df, C9, C10):
    getBinValues(df,C9, C10) 
    expandObservableData(df)

def getObservablesByName(DATADIR, EXPDIR, names):
    names = [f for f in listdir(DATADIR) if isfile(join(DATADIR, f)) and f.split('.')[0] in names]
    return getObservables(DATADIR, EXPDIR, names)     

    
def computeAllModelsAllFit():
    DATADIR = 'data/models/observables' 
    EXPDIR = 'data/experimental'  
    models = getModels()
    m = getCoeffsFromModels(models)
    observables = getAllObservables()
    #observables = getObservablesByName(DATADIR, EXPDIR, ['P5'])
    return computeModelsFit(m, observables)

def getValue(value, low, high ):    
    #print('value: {}, high: {}, low: {}, newValue: {}'.format(value,high,low, random.uniform(value - low, value + high)))
    return np.random.normal(value, low)

def generateExperimental():    
    observables = loadObservables()
    x = np.array([])
    for mo, eo in observables:
        dfe = eo['data']
        x = np.append(x, dfe['BinValue'].values)
        
    saveData("X_experimental.pickle", x)
    return x

def saveData(filename, data):
    pickle_out = open(filename, "wb")
    pickle.dump(data, pickle_out)
    pickle_out.close()         

    
def generateBinErrors():
    observables = loadObservables() 
    
    errors = np.empty([0,6]);
    for mo, eo in observables:
        
        dfm = mo['data']
        dfe = eo['data']
        expandObservableData(dfm)
        expandObservableData(dfe)
        dfmErrorMinus = dfm['ErrorMinus']  
        dfeErrorMinus = dfe['ErrorMinus']  
        dfmError = dfm['Heights'].values / 2
        dfeError = dfe['Heights'].values / 2
        widths = dfe['Widths'].values
        bins = dfm['BinLow'].values
        errors = np.append(errors, [[a,b,c,d,e,f] for a,b,c,d,e,f in zip(dfmError,dfeError,widths,bins,dfmErrorMinus,dfeErrorMinus)],axis=0)
     
    saveData("errors.pickle", errors)
    

def generateObservableNumBins():
    observables = loadObservables() 
    
    numBins = []
    for mo, eo in observables:
        dfe = eo['data']
        numBins.append(len(dfe.index))
     
    saveData("numBins.pickle", numBins)
    print(numBins)


    
def genNNData():
    observables = loadObservables()  
    models = getRandomModels()
   
    X = []
    Y = []
    coeffs = []
    iter=0   
    bestFit=100000
    bestc=[]
    allFits=[]
    for c9, c10 in models:
        print(iter)
        iter=iter+1             
            
       
        x = []
        y = np.array([])

        fits=[]        

        for mo, eo in observables:
            dfm = mo['data']
            dfe = eo['data']                

            computeModelObservable(dfm,c9, c10)


            fit = computeModelFit(dfm, dfe)
            fits.append(fit)                

            x = x + dfm['BinValue'].values.tolist()          


        y = np.append(y,[c9,c10])     
       
        fit = np.array(fits).sum()
        allFits.append(fit)
     
        if(fit<bestFit):
            bestFit=fit
            bestc = [c9,c10]
            print(bestc)
            print(fit)
            
        # FOR GAN -----------------------------
        x = x + [c9,c10]
        #--------------------------------------
        X.append(x)
        
        Y.append(y.tolist()) 
            
        
    
    generateExperimental()
    X = np.array(X)
    Y = np.array(Y)  
    
    
    saveData("X_NN.pickle", X)
    saveData("y_NN.pickle", Y)
    saveData("histLogChisquare.pickle", allFits)
    
    print(bestFit)
    print(bestc)
    print(X[0])
    
    
def genNNWithCoeffsData():
    observables = loadObservables()  
    models = getRandomModels()
   
    X = []
    Y = []
    coeffs = []
    numPolyCoeffs = 100
    iter=0   
    bestFit=100000
    bestq=[]
    bestc=[]
    for c9, c10 in models:
        print(iter)
        iter=iter+1        
        
        for i in range(numPolyCoeffs):  
            
            allFits = []      
            x = []
            y = np.array([])
            allQSquareCoeffs = np.array([])            
            fits=[]
            numObs = 0
            
            for mo, eo in observables:
                dfm = mo['data']
                dfe = eo['data']                
                
                computeModelObservable(dfm,c9, c10)
                
                if(i==0):
                    QSsquareCoeffs = np.random.uniform(0, 0, 3)                
                else:
                    QSsquareCoeffs = [rnd() for _ in range(3)]
                    QSsquareCoeffs[1] = QSsquareCoeffs[1]/10
                    QSsquareCoeffs[2] = QSsquareCoeffs[2]/100
                    QSsquareCoeffs = np.round(QSsquareCoeffs,2)
                  
                
                allQSquareCoeffs = np.append(allQSquareCoeffs, QSsquareCoeffs)                
                addQPoly(dfm, QSsquareCoeffs)                
                fit = computeModelFit(dfm, dfe)
                fits.append(fit)                
                
                x = x + dfm['BinValue'].values.tolist()
                numObs = numObs + 1
                
            
            y = np.append(y,[c9,c10])     
            y = np.append(y, allQSquareCoeffs)
            fit = np.array(fits).sum()
            if(fit<bestFit):
                bestFit=fit
                bestc = [c9,c10]
                bestq = allQSquareCoeffs
                print(bestc)
                print(bestq)
                print(fit)
            X.append(x)
            #X.append(x + [fit])
            
            
            Y.append(y.tolist()) 
            
        
    
    generateExperimental()
    X = np.array(X)
    Y = np.array(Y)  
    
    
    saveData("X_NN_With_Coeffs.pickle", X)
    saveData("y_NN_With_Coeffs.pickle", Y)
    
    saveData("coeffs_noChi.pickle", coeffs)  
    print(bestFit)
    print(bestc)
    print(bestq)
    

    
def loadObservables():
    DATADIR = 'data/models/observables' 
    EXPDIR = 'data/experimental'  
    observables = []
    
    P1 = getObservablesByName(DATADIR, EXPDIR, ['P1'])
    P2 = getObservablesByName(DATADIR, EXPDIR, ['P2'])
    P4 = getObservablesByName(DATADIR, EXPDIR, ['P4'])
    P5 = getObservablesByName(DATADIR, EXPDIR, ['P5'])
    BRK0Starmumu = getObservablesByName(DATADIR, EXPDIR, ['BRK0Starmumu'])
    BRK0mumu = getObservablesByName(DATADIR, EXPDIR, ['BRK0mumu'])
    RK = getObservablesByName(DATADIR, EXPDIR, ['RK'])
    RKStar = getObservablesByName(DATADIR, EXPDIR, ['RKStar'])    
    #
    observables = np.array([P1, P2, P4, P5, BRK0Starmumu, BRK0mumu, RK, RKStar])
    observables = observables.reshape((8,2))
    
    #observables = np.array([RKStar])
    #observables = observables.reshape((1,2))
    
    
    #observables = np.array([P1])
    #observables = observables.reshape((1,2))
    return observables

    
def getRandomModels():
    #c9 = np.round(np.linspace(-2, 0, num=10), 2)
    #c10 = np.round(np.linspace(-1, 1, num=10), 2)    
    c9 = np.round(np.linspace(-5, 5, num=50), 2)
    c10 = np.round(np.linspace(-5, 5, num=50), 2)    
    models = np.meshgrid(c9, c10)    
    
    r = np.array([(x,y) for a,b in zip(models[0], models[1]) for x,y in zip(a,b)]) 
    #r = np.array([(x,y) for x,y in zip(c9,c10)])      
    area = np.pi*2
    # Plot
    plt.scatter(r[:,0], r[:,1], s=area, alpha=1)
    plt.show() 
    return r   


    
def rnd():
    exp = random.randint(-19, -15)
    base = 10 * random.random() - 1
    #return base * 10**exp
    #return random.random()
    return round(random.uniform(-1, 1),2)
    return round(random.uniform(-0.010, 0.010),2)
    


def computeModelCoeffsFit(coeffs, q2coeffs):
    observables = loadObservables() 
    c9, c10 = coeffs[0]
    fits=[]
    numObs = 0
    for mo, eo in observables:
        dfm = mo['data']
        dfe = eo['data'] 
        computeModelObservable(dfm,c9, c10)

        if(len(q2coeffs) > 0):            
            addQPoly(dfm, q2coeffs[numObs])

        fit = computeModelFit2(dfm, dfe)
        
        fits.append(fit)
        numObs = numObs+1
        
    return np.array(fits).sum()

In [ ]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop


import keras.backend as K

import matplotlib.pyplot as plt

import sys

import numpy as np
d_hist = []
g_hist = []
d_accu = []
class WGAN():
    def __init__(self):
        self.img_rows = 8
        self.img_cols = 8
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        self.clip_value = 0.01
        optimizer = RMSprop(lr=0.00005)

        # Build and compile the critic
        self.critic = self.build_critic()
        self.critic.compile(loss=self.wasserstein_loss,
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.critic.trainable = False

        # The critic takes generated images as input and determines validity
        valid = self.critic(img)

        # The combined model  (stacked generator and critic)
        self.combined = Model(z, valid)
        self.combined.compile(loss=self.wasserstein_loss,
            optimizer=optimizer,
            metrics=['accuracy'])

    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(Conv2D(128, kernel_size=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(64, kernel_size=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=2, padding="same"))
        

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_critic(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        #model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        #model.add(BatchNormalization(momentum=0.8))
        #model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        #model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        #model.add(BatchNormalization(momentum=0.8))
        #model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        #model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        #model.add(BatchNormalization(momentum=0.8))
        #model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        pickle_in = open("X_NN_With_Coeffs.pickle", "rb")
        X_train = pickle.load(pickle_in)
        X_train = np.reshape(X_train, (-1,8,8))

        # Rescale -1 to 1
        
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake = np.ones((batch_size, 1))

        for epoch in range(epochs):

            for _ in range(self.n_critic):

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Select a random batch of images
                idx = np.random.randint(0, X_train.shape[0], batch_size)
                imgs = X_train[idx]
                
                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                # Train the critic
                d_loss_real = self.critic.train_on_batch(imgs, valid)
                d_loss_fake = self.critic.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)

                # Clip critic weights
                for l in self.critic.layers:
                    weights = l.get_weights()
                    weights = [np.clip(w, -self.clip_value, self.clip_value) for w in weights]
                    l.set_weights(weights)


            # ---------------------
            #  Train Generator
            # ---------------------

            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f] [G loss: %f]" % (epoch, 1 - d_loss[0], 1 - g_loss[0]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.save_imgs(epoch, d_loss, g_loss)

    def save_imgs(self, epoch, d_loss, g_loss):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        predictions = np.reshape(gen_imgs[0],(64))
        print(predictions.shape)
        realfit = computeModelCoeffsFit([predictions[37:39]],np.reshape(predictions[39:-1],(8,3)))
        print(realfit)
        print(predictions[37:])
        # Rescale images 0 - 1
        #gen_imgs = 0.5 * gen_imgs + 0.5
        self.generator.save('generator')
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        d_hist.append(d_loss[0])
        d_accu.append(d_loss[1])
        g_hist.append(g_loss[0])
        fig, ax = plt.subplots(1,2)
        fig.set_figheight(7)
        fig.set_figwidth(15)
        ax[0].set_xlabel(r'Iteration', fontsize=16)
        ax[0].set_ylabel(r'Loss', fontsize=16)
        ax[1].set_xlabel(r'Iteration', fontsize=16)
        ax[1].set_ylabel(r'Accuracy', fontsize=16)
        plt.xticks(fontsize=14)
        p1 = ax[0].plot(g_hist)    
        p2 = ax[0].plot(d_hist)
        p3 = ax[1].plot(d_accu, color='darkorange')

        ax[0].legend((p1[0], p2[0]), ('Generator', 'Discriminator'))
        
        plt.savefig('gendis/gendis3.svg')
        plt.show()               
        plt.close()


if __name__ == '__main__':
    wgan = WGAN()
    wgan.train(epochs=4000, batch_size=32, sample_interval=50)

In [ ]:
from tensorflow import keras
import sys
import numpy as np
import pickle
import matplotlib.pyplot as plt 
np.set_printoptions(threshold=sys.maxsize)



model = keras.models.load_model('generator')
chi2=[]
plt.rcParams.update({'font.size': 22})
latent_space =[]

def saveData(filename, data):
    pickle_out = open(filename, "wb")
    pickle.dump(data, pickle_out)
    pickle_out.close()  

numSamples = 10000
dim = 100
noise = np.random.normal(0, 1, (numSamples, dim))

gen_imgs = model.predict(noise)


gen_imgs = np.reshape(gen_imgs,(numSamples, 64))

bestCoeff=[]
bestFit=999999
for i in range(numSamples):
    print(i)
    
    predictions = gen_imgs[i]
    #v = computeModelFit(predictions,X_experimental,modelErrors,expErrors)
    realfit = computeModelCoeffsFit([predictions[37:39]],np.reshape(predictions[39:-1],(8,3)))
    #realfit = computeModelCoeffsFit([predictions[37:39]],[])
    
    chi2.append(np.round(realfit,1))
    if(realfit<bestFit):
        print(realfit)
        print(predictions)
        bestFit = realfit
        
    



label = np.sort(chi2)[0]

print(label)



    


In [ ]:
from sklearn.preprocessing import StandardScaler
def print_imgs():
    r, c = 5, 5   
    pickle_in = open("X_NN_With_Coeffs.pickle", "rb")
    X_train = pickle.load(pickle_in)
    print(X_train[])
   #X_train = np.reshape(X_train, (-1,8,8))
   #idx = np.random.randint(0, 10000, r*c)

   #gen_imgs = X_train[idx]
   #print(gen_imgs[0])
   ## Rescale images 0 - 1
   ##gen_imgs = 0.5 * gen_imgs + 0.5        
   #fig, axs = plt.subplots(r, c)
   #cnt = 0
   #for i in range(r):
   #    for j in range(c):
   #        axs[i,j].imshow(gen_imgs[cnt, :,:], cmap='gray')
   #        axs[i,j].axis('off')
   #        cnt += 1
   #plt.savefig('images/trainingimages.svg')
   #
   #plt.show()               
   #plt.close()
    
#print_imgs()